# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

#### 1.1. Importing the Data

We first import the relevant packages.

In [9]:
import glob
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

We then import the files of the three countries each in a `Dataframe`.

In [10]:
def import_data(country):
    """Returns Dataframe resulting from concatenation of files for given country"""
    file_list = glob.glob('Data/ebola/{}_data/*.csv'.format(country))
    return pd.concat([pd.read_csv(file_) for file_ in file_list])

guinea = import_data("guinea")
liberia = import_data("liberia")
sl = import_data("sl")

#### 1.2. Finding Data Correspondences

Reading the different dataframes, we see that there are five categories of interest if we want to get the daily average per month of **new** cases and deaths.

In [11]:
CASES_SUSPECTS = "New Cases: Suspects"
CASES_PROBABLES = "New Cases: Probables"
CASES_CONFIRMED = "New Cases: Confirmed"
CASES_TOTAL = "New Cases: Total"
DEATHS_TOTAL = "New Deaths: Total"

Then, we can map the correspondences from the files to these categories in the following functions.

In [12]:
def label_guinea(description):
    try:
        return {"New cases of suspects": CASES_SUSPECTS,
                "New cases of probables": CASES_PROBABLES,
                "New cases of confirmed": CASES_CONFIRMED,
                "Total new cases registered so far": CASES_TOTAL,
                "New deaths registered today": DEATHS_TOTAL,
                "New deaths registered": DEATHS_TOTAL}[description]
    except:
        return None

def label_liberia(description):
    try:
        return {"New Case/s (Suspected)": CASES_SUSPECTS,
                "New Case/s (Probable)": CASES_PROBABLES,
                "New case/s (confirmed)": CASES_CONFIRMED,
                "Newly reported deaths": DEATHS_TOTAL}[description]
    except:
        return None

def label_sl(description):
    try:
        return {"new_suspected": CASES_SUSPECTS,
                "new_probable": CASES_PROBABLES,
                "new_confirmed": CASES_CONFIRMED,
                "etc_new_deaths": DEATHS_TOTAL}[description]
    except:
        return None

#### 1.3. Exploiting and Merging the Data

We first establish a function that gives the `DataFrame` corresponding to each country's data summary: it contains the daily average per month and per year of new cases and deaths.

In [13]:
def get_summary_dataframe(data, country, number_column, text_column, date_column, label_function):
    """
    Gives DataFrame containing the daily average per month and per year of the given country's new cases and deaths.
    :param data: DataFrame containing the imported data from files
    :param country: string, name of the country
    :param number_column: string, name of the column containing the numbers to be averaged
    :param text_column: string, name of the column containing the description of the numbers given
    :param date_column: string, name of the column containing the date of the record
    :param label_function: function, corresponding to function mapping files to data categories
    :return: DataFrame
    """
    df_copy = data.copy()
    
    # Applying the mapping and filtering
    df_copy[text_column] = df_copy[text_column].apply(label_function)
    df_copy = df_copy[(df_copy[text_column].notnull())]
    
    # Filtering the columns
    df = df_copy[[text_column, number_column]]
    
    # Making sure all of the data in the number column are interpreted as integers, with NaNs replaced by zero
    df[number_column] = df[number_column].fillna(0).apply(int)
    
    # Renaming the column corresponding to text_column
    df["Daily Average"] = df[text_column].rename("Daily Average")
    
    # Adding Country Column
    df['Country'] = pd.Series([country] * len(df), index=df.index)
    
    # Adding columns to split the Date
    df['Year'] = pd.DatetimeIndex(df_copy[date_column]).year
    df['Month'] = pd.DatetimeIndex(df_copy[date_column]).month
    df['Day'] = pd.DatetimeIndex(df_copy[date_column]).day
    
    # Reshaping the DataFrame
    df = df.set_index(["Daily Average", "Country", 'Year', 'Month', 'Day'])[number_column].unstack("Daily Average")
    
    # Averaging
    df = df.mean(level=[0,1,2])
    return df

By reading the columns of the `DataFrame` instances, we can establish the names of the columns of interest. Then, we use the function above to get the summary data frames and we concatenate them. The result is shown below.

In [14]:
df_guinea = get_summary_dataframe(guinea, "Guinea", "Totals", "Description", "Date", label_guinea)
df_liberia = get_summary_dataframe(liberia, "Liberia", "National", "Variable", "Date", label_liberia)
df_sl = get_summary_dataframe(sl, "Sierra Leone", "National", "variable", "date", label_sl)
pd.concat([df_guinea, df_liberia, df_sl])

New Cases: Confirmed  New Cases: Probables  \
Country      Year Month                                               
Guinea       2014 8                 12.400000              1.600000   
                  9                 13.000000              1.187500   
                  10                 6.000000              0.000000   
Liberia      2014 6                  2.142857              1.142857   
                  7                  1.818182              3.727273   
                  8                  5.444444             19.777778   
                  9                  6.166667             29.333333   
                  10                 1.360000             17.760000   
                  11                 2.600000              7.000000   
                  12              1928.333333           1208.000000   
Sierra Leone 2014 8                 18.550000              1.150000   
                  9                 34.413793              0.000000   
                  10                56.714286              0.857143   
                  11                59.904762              0.000000   
                  12                32.600000              0.000000   

                         New Cases: Suspects  New Cases: Total  \
Country      Year Month                                          
Guinea       2014 8                11.800000            25.800   
                  9                 5.437500            19.625   
                  10               28.000000            34.000   
Liberia      2014 6                 2.428571               NaN   
                  7                 3.000000               NaN   
                  8                12.000000               NaN   
                  9                28.333333               NaN   
                  10               26.440000               NaN   
                  11               16.866667               NaN   
                  12             2042.222222               NaN   
Sierra Leone 2014 8                 5.450000               NaN   
                  9                 6.275862               NaN   
                  10               13.357143               NaN   
                  11               15.333333               NaN   
                  12                8.400000               NaN   

                         New Deaths: Total  
Country      Year Month                     
Guinea       2014 8               3.400000  
                  9               3.562500  
                  10             15.000000  
Liberia      2014 6               2.000000  
                  7               4.272727  
                  8              23.222222  
                  9              36.041667  
                  10             28.040000  
                  11             13.466667  
                  12              0.000000  
Sierra Leone 2014 8               0.000000  
                  9               0.275862  
                  10              3.535714  
                  11              0.571429  
                  12              2.200000

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

#### 2.1. Importing the Data

We first import the nine spreadsheets into a single `DataFrame`.

In [15]:
# Folder path and list of file paths to import
path = 'Data/microbiome/'
all_files = glob.glob(path +  "MID*.xls")

# Iterating over the files
microbiome_dfs = []
for file in all_files:
    microbiome_file = pd.read_excel(file, header=None)
    microbiome_file['BARCODE'] = file.split(path)[1].split(".")[0]
    microbiome_dfs.append(microbiome_file)

# Concatenate the data frames
microbiome = pd.concat(microbiome_dfs, ignore_index = True)

# Preview
microbiome.head()

,0,1,BARCODE
0,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",7,MID1
1,"Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2,MID1
2,"Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3,MID1
3,"Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3,MID1
4,"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7,MID1


Then, we import the metadata file.

In [16]:
metadata = pd.read_excel(path + 'metadata.xls')
metadata.head()

,BARCODE,GROUP,SAMPLE
0,MID1,EXTRACTION CONTROL,NaN
1,MID2,NEC 1,tissue
2,MID3,Control 1,tissue
3,MID4,NEC 2,tissue
4,MID5,Control 2,tissue


#### 2.2. Merging the Data

We first merge the two `DataFrame` instances from above and fill the `NaN` values with the `unknown` tag. Then, we set up a unique index.

In [17]:
# Merging and Filling in NaN values
merged_df = pd.merge(microbiome, metadata).fillna('unknown')

# Renaming columns and setting indexes
merged_df = merged_df.rename(columns={0: 'TAXON', 1: 'COUNT'})
merged_df = merged_df.set_index(['BARCODE', 'TAXON'])[['GROUP', 'SAMPLE', 'COUNT']]

# Preview
merged_df.head()

GROUP  \
BARCODE TAXON                                                                    
MID1    Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Sulfolobal...  EXTRACTION CONTROL   
        Archaea "Crenarchaeota" Thermoprotei Thermoprot...  EXTRACTION CONTROL   
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...  EXTRACTION CONTROL   

                                                             SAMPLE  COUNT  
BARCODE TAXON                                                               
MID1    Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  unknown      7  
        Archaea "Crenarchaeota" Thermoprotei Desulfuroc...  unknown      2  
        Archaea "Crenarchaeota" Thermoprotei Sulfolobal...  unknown      3  
        Archaea "Crenarchaeota" Thermoprotei Thermoprot...  unknown      3  
        Archaea "Euryarchaeota" "Methanomicrobia" Metha...  unknown      7

We verify that the index is unique.

In [18]:
merged_df.index.is_unique

True

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here